In [576]:
import itertools

import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.cp_tensor import cp_to_tensor
import matplotlib.pyplot as plt
import random
np.random.seed(42)

In [577]:
# Parameters

alpha = 10000 # Varying parameter: Influence parameter
lambda_ = 180 # Varying parameter: Drift parameter
sigma = 10 # Varying parameter: Diffusion parameter
dt = [0.01] # Varying parameter: Time step
n_particles = [100, 150] # Varying parameter: Number of particles, number of initial random tensors
T = 1.0 # Fixed parameter: Total time
#n_iterations = int(T / dt) # Varying parameter: Number of iterations

# Create itertools.product to generate all combinations of parameters
# param_grid = list(itertools.product(dt, n_particles, alpha, lambda_, sigma))

I, J, K = 3, 3, 3 # Fixed parameters: Dims of initial tensor
rank = 2 # Fixed parameter: number of rank-1 tensors

# Create a random tensor (change np.random.seed(--) in case of need)
tensor = tl.tensor(np.random.random((I, J, K)))

In [578]:
tensor

array([[[0.37454012, 0.95071431, 0.73199394],
        [0.59865848, 0.15601864, 0.15599452],
        [0.05808361, 0.86617615, 0.60111501]],

       [[0.70807258, 0.02058449, 0.96990985],
        [0.83244264, 0.21233911, 0.18182497],
        [0.18340451, 0.30424224, 0.52475643]],

       [[0.43194502, 0.29122914, 0.61185289],
        [0.13949386, 0.29214465, 0.36636184],
        [0.45606998, 0.78517596, 0.19967378]]])

In [579]:
# particles[i], where i = [A_i, B_i, C_i] and A_i.shape = 10x5, B_i.shape = 12x5, C_i.shape = 14x5
# a{r}_{i} = particles[i-1]['A'][:,r-1]
# where r = rank from 1 to 5, i = particles from 1 to n_particles

"""
# example indexing:
a1_1 = particles[0]['A'][:,0]
b1_1 = particles[0]['B'][:,0]
c1_1 = particles[0]['C'][:,0]

a2_1 = particles[0]['A'][:,1]
b2_1 = particles[0]['B'][:,1]
c2_1 = particles[0]['C'][:,1]

a3_1 = particles[0]['A'][:,2]
b3_1 = particles[0]['B'][:,2]
c3_1 = particles[0]['C'][:,2]

a4_1 = particles[0]['A'][:,3]
b4_1 = particles[0]['B'][:,3]
c4_1 = particles[0]['C'][:,3]

a5_1 = particles[0]['A'][:,4]
b5_1 = particles[0]['B'][:,4]
c5_1 = particles[0]['C'][:,4]

# ----------------------------

a1_2 = particles[1]['A'][:,0]
b1_2 = particles[1]['B'][:,0]

# and so on"""

"\n# example indexing:\na1_1 = particles[0]['A'][:,0]\nb1_1 = particles[0]['B'][:,0]\nc1_1 = particles[0]['C'][:,0]\n\na2_1 = particles[0]['A'][:,1]\nb2_1 = particles[0]['B'][:,1]\nc2_1 = particles[0]['C'][:,1]\n\na3_1 = particles[0]['A'][:,2]\nb3_1 = particles[0]['B'][:,2]\nc3_1 = particles[0]['C'][:,2]\n\na4_1 = particles[0]['A'][:,3]\nb4_1 = particles[0]['B'][:,3]\nc4_1 = particles[0]['C'][:,3]\n\na5_1 = particles[0]['A'][:,4]\nb5_1 = particles[0]['B'][:,4]\nc5_1 = particles[0]['C'][:,4]\n\n# ----------------------------\n\na1_2 = particles[1]['A'][:,0]\nb1_2 = particles[1]['B'][:,0]\n\n# and so on"

In [580]:
def objective_function(particle):
    # Reconstruct tensor from the given particle
    A = particle['A']
    B = particle['B']
    C = particle['C']
    reconstructed_tensor = cp_to_tensor((np.ones(rank), [A, B, C]))

    # Compute the Frobenius norm of the difference
    error = tl.norm(tensor - reconstructed_tensor)

    return error

In [581]:
def compute_consensus_point(particles, alpha):
    # Find the particle with the minimum energy (x*)
    min_energy_particle = min(particles, key=objective_function)
    min_energy = objective_function(min_energy_particle)

    # Initialize the numerators and the denominator
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle
        energy = objective_function(particle)

        # Apply the numerical stabilization trick
        weight = np.exp(-alpha * (energy - min_energy))

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Compute the consensus matrices (now no need for epsilon)
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}


In [582]:
"""def compute_consensus_point(particles, alpha):
    numerator_A = np.zeros_like(particles[0]['A'])
    numerator_B = np.zeros_like(particles[0]['B'])
    numerator_C = np.zeros_like(particles[0]['C'])
    denominator = 0.0

    for particle in particles:
        A, B, C = particle['A'], particle['B'], particle['C']

        # Compute the energy for the current particle (Frobenius norm error)
        energy = objective_function(particle)

        # Compute the weight for this particle
        weight = np.exp(-alpha * energy)

        # Update the numerators and the denominator
        numerator_A += weight * A
        numerator_B += weight * B
        numerator_C += weight * C
        denominator += weight

    # Add a small epsilon to avoid division by zero
    epsilon = 1e-10
    denominator = np.maximum(denominator, epsilon)

    # Compute the consensus matrices
    consensus_A = numerator_A / denominator
    consensus_B = numerator_B / denominator
    consensus_C = numerator_C / denominator

    return {'A': consensus_A, 'B': consensus_B, 'C': consensus_C}"""

"def compute_consensus_point(particles, alpha):\n    numerator_A = np.zeros_like(particles[0]['A'])\n    numerator_B = np.zeros_like(particles[0]['B'])\n    numerator_C = np.zeros_like(particles[0]['C'])\n    denominator = 0.0\n\n    for particle in particles:\n        A, B, C = particle['A'], particle['B'], particle['C']\n\n        # Compute the energy for the current particle (Frobenius norm error)\n        energy = objective_function(particle)\n\n        # Compute the weight for this particle\n        weight = np.exp(-alpha * energy)\n\n        # Update the numerators and the denominator\n        numerator_A += weight * A\n        numerator_B += weight * B\n        numerator_C += weight * C\n        denominator += weight\n\n    # Add a small epsilon to avoid division by zero\n    epsilon = 1e-10\n    denominator = np.maximum(denominator, epsilon)\n\n    # Compute the consensus matrices\n    consensus_A = numerator_A / denominator\n    consensus_B = numerator_B / denominator\n    con

In [583]:
# Additional projection operator for the hypersurface and the special case hypersphere
def projection_operator_matrix(V):
    """Apply the projection operator P(V) = I - (V ⊗ V) / |V|^2 to each column of the matrix V."""
    V_norm_sq = np.sum(V**2, axis=0, keepdims=True)  # Compute |V|^2 for each column (1 x n_columns)
    outer_product = np.einsum('ik,jk->ijk', V, V)  # Compute V ⊗ V for each column (n_rows x n_rows x n_columns)
    I = np.eye(V.shape[0])  # Identity matrix of appropriate size (n_rows x n_rows)
    P = I[:, :, np.newaxis] - outer_product / V_norm_sq  # Apply the projection operator column-wise
    return P

In [584]:
# Create random normal values between 0 and 1 for the Brownıan motion

def truncated_normal_samples(size, mean=0, std=1, lower=0, upper=1):
    # size can be a tuple like (3,4)
    total_samples = np.prod(size)  # total number of elements
    samples = np.empty(total_samples)
    count = 0
    while count < total_samples:
        batch = np.random.normal(loc=mean, scale=std, size=total_samples - count)
        batch = batch[(batch >= lower) & (batch <= upper)]
        samples[count:count+len(batch)] = batch
        count += len(batch)
    # Reshape the samples to the desired shape
    return samples.reshape(size)

#arr_truncated = truncated_normal_samples(size=(3,4), mean=0, std=1, lower=0, upper=1)

In [585]:
def isotropic_update(particles, consensus_point, lambda_, sigma, dt):
    # Compute the consensus loss using the given consensus_point
    consensus_point_loss = objective_function(consensus_point)

    for particle in particles:
        """
        Perform isotropic updates on all particles.

        Parameters:
        - particles: list of particle dictionaries with keys 'A', 'B', 'C'.
        - consensus_point: dictionary with keys 'A', 'B', 'C' representing the consensus matrices.
        - lambda_: drift parameter.
        - sigma: diffusion parameter.
        - dt: time step.
        """

        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        if consensus_point_loss < objective_function(particle):
            drift_A = (-lambda_) * (A - A_consensus) * dt # I x r
            drift_B = (-lambda_) * (B - B_consensus) * dt # J x r
            drift_C = (-lambda_) * (C - C_consensus) * dt # K x r
        else:
            drift_A = np.zeros_like(A) * dt # I x r
            drift_B = np.zeros_like(B) * dt # J x r
            drift_C = np.zeros_like(C) * dt # K x r

        # Create independent Gaussian noise for each element of A, B, C: Brownian motion
        B_A = truncated_normal_samples(size=A.shape, mean=0, std=1, lower=0, upper=1)
        B_B = truncated_normal_samples(size=B.shape, mean=0, std=1, lower=0, upper=1)
        B_C = truncated_normal_samples(size=C.shape, mean=0, std=1, lower=0, upper=1)

        # Isotropic noise term, applied column_wise
        noise_A = sigma * np.linalg.norm(A - A_consensus) * B_A * np.sqrt(dt)
        noise_B = sigma * np.linalg.norm(B - B_consensus) * B_B * np.sqrt(dt)
        noise_C = sigma * np.linalg.norm(C - C_consensus) * B_C * np.sqrt(dt)

        # Update particle's A, B, and C matrices
        particle['A'] += np.array(drift_A) + np.array(noise_A)
        particle['B'] += np.array(drift_B) + np.array(noise_B)
        particle['C'] += np.array(drift_C) + np.array(noise_C)

    return particles


In [586]:
def anisotropic_update(particles, consensus_point, lambda_, sigma, dt):
    # Compute the consensus loss using the given consensus_point
    consensus_point_loss = objective_function(consensus_point)

    for particle in particles:
        # Extract the matrices A, B, C from the particle and the consensus
        A, B, C = particle['A'], particle['B'], particle['C']
        A_consensus, B_consensus, C_consensus = consensus_point['A'], consensus_point['B'], consensus_point['C']

        if consensus_point_loss < objective_function(particle):
            drift_A = (-lambda_) * (A - A_consensus) * dt # I x r
            drift_B = (-lambda_) * (B - B_consensus) * dt # J x r
            drift_C = (-lambda_) * (C - C_consensus) * dt # K x r
        else:
            drift_A = np.zeros_like(A) * dt # I x r
            drift_B = np.zeros_like(B) * dt # J x r
            drift_C = np.zeros_like(C) * dt # K x r

        # Create independent Gaussian noise for each element of A, B, C: Brownian motion
        B_A = truncated_normal_samples(size=A.shape, mean=0, std=1, lower=0, upper=1)
        B_B = truncated_normal_samples(size=B.shape, mean=0, std=1, lower=0, upper=1)
        B_C = truncated_normal_samples(size=C.shape, mean=0, std=1, lower=0, upper=1)

        # Anisotropic diffusion terms
        diffusion_A = sigma * (A - A_consensus) * B_A * np.sqrt(dt)
        diffusion_B = sigma * (B - B_consensus) * B_B * np.sqrt(dt)
        diffusion_C = sigma * (C - C_consensus) * B_C * np.sqrt(dt)

        # Update particle's A, B, and C matrices
        particle['A'] += drift_A + diffusion_A
        particle['B'] += drift_B + diffusion_B
        particle['C'] += drift_C + diffusion_C

    return particles

In [587]:
particles = []
nu=1000
n_iterations=100
for _ in range(nu):
    A = np.random.randn(I, rank) # n_particles Ixrank matrices
    B = np.random.randn(J, rank) # n_particles Jxrank matrices
    C = np.random.randn(K, rank) # n_particles Kxrank matrices
    particles.append({'A': A, 'B': B, 'C': C})

In [588]:
# Main iteration loop
for iteration in range(n_iterations):
    # Step 1: Calculate the consensus point
    consensus_point = compute_consensus_point(particles, alpha=alpha)

    # Step 2: Compute the reconstruction error for the consensus point
    consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
    consensus_error = tl.norm(tensor - consensus_tensor)

    # Print the reconstruction error of the consensus point
    print(f"Iteration {iteration + 1}/{n_iterations}, Consensus Reconstruction Error: {consensus_error}")

    # Step 3: Update the particles (either isotropic or anisotropic)
    particles = anisotropic_update(particles, consensus_point, lambda_, sigma, dt)


Iteration 1/100, Consensus Reconstruction Error: 2.648223233871237
Iteration 2/100, Consensus Reconstruction Error: 1.9524582617252402
Iteration 3/100, Consensus Reconstruction Error: 1.7076532271240368
Iteration 4/100, Consensus Reconstruction Error: 1.4845644324511644
Iteration 5/100, Consensus Reconstruction Error: 1.3317847158715823
Iteration 6/100, Consensus Reconstruction Error: 1.2558100548196232
Iteration 7/100, Consensus Reconstruction Error: 1.215804659662353
Iteration 8/100, Consensus Reconstruction Error: 1.1887013027415072
Iteration 9/100, Consensus Reconstruction Error: 1.1753947385662278
Iteration 10/100, Consensus Reconstruction Error: 1.167880356357271
Iteration 11/100, Consensus Reconstruction Error: 1.1642397002468834
Iteration 12/100, Consensus Reconstruction Error: 1.1617327452236874
Iteration 13/100, Consensus Reconstruction Error: 1.160315928620487
Iteration 14/100, Consensus Reconstruction Error: 1.159520044419672
Iteration 15/100, Consensus Reconstruction Error

In [589]:
"""results = []
# Create itertools.product to generate all combinations of parameters
param_grid = list(itertools.product(dt, n_particles, alpha, lambda_, sigma))

# Iterate over the parameter grid
for params in param_grid:
    dt, n_particles, alpha, lambda_, sigma = params

    n_iterations = int(T / dt) # Varying parameter: Number of iterations

    # Step 2: Initialize
    particles = []
    for _ in range(n_particles):
        A = np.random.randn(I, rank) # n_particles Ixrank matrices
        B = np.random.randn(J, rank) # n_particles Jxrank matrices
        C = np.random.randn(K, rank) # n_particles Kxrank matrices
        particles.append({'A': A, 'B': B, 'C': C})

    # Print all parameters including fixed ones
    print(f"Running grid search with params: dt={dt}, n_particles={n_particles}, alpha={alpha}, lambda_={lambda_}, sigma={sigma}, T={T}, I={I}, J={J}, K={K}, rank={rank}, #iters={n_iterations}")

    # List to store reconstruction errors of the consensus point at each iteration
    consensus_errors = []

    # Main iteration loop
    for iteration in range(n_iterations):
        # Step 1: Calculate the consensus point
        consensus_point = compute_consensus_point(particles, alpha=alpha)

        # Step 2: Compute the reconstruction error for the consensus point
        consensus_tensor = cp_to_tensor((np.ones(rank), [consensus_point['A'], consensus_point['B'], consensus_point['C']]))
        consensus_error = tl.norm(tensor - consensus_tensor)

        # Print the reconstruction error of the consensus point
        print(f"Iteration {iteration + 1}/{n_iterations}, Consensus Reconstruction Error: {consensus_error}")

        # Store the error for later plotting
        consensus_errors.append(consensus_error)

        # Step 3: Update the particles (either isotropic or anisotropic)
        anisotropic_update(particles, consensus_point, lambda_, sigma)

    # Collect errors at specific iterations: 1st, 1/4th, 1/2th, 3/4th, and last iteration
    error_begin = consensus_errors[0]
    error_iter_1_4 = consensus_errors[int(n_iterations * 0.25)]
    error_iter_1_2 = consensus_errors[int(n_iterations * 0.5)]
    error_iter_3_4 = consensus_errors[int(n_iterations * 0.75)]
    error_last = consensus_errors[-1]

    # Add the parameters and errors to the results list
    results.append({
        'alpha': alpha,
        'lambda': lambda_,
        'sigma': sigma,
        'T': T,
        'dt': dt,
        '#iters (T/dt)': n_iterations,
        '#particles': n_particles,
        'I': I,
        'J': J,
        'K': K,
        'rank': rank,
        'error_begin': error_begin,
        'error_iter 1/4': error_iter_1_4,
        'error_iter 1/2': error_iter_1_2,
        'error_iter 3/4': error_iter_3_4,
        'error_iter last': error_last,
        'landscape': ''  # Fill this field if needed
    })

    # Final output: Save or analyze the final consensus point
    final_consensus_point = compute_consensus_point(particles, alpha=alpha)
    final_consensus_tensor = cp_to_tensor((np.ones(rank), [final_consensus_point['A'], final_consensus_point['B'], final_consensus_point['C']]))
    final_error = tl.norm(tensor - final_consensus_tensor)

    print("Final Consensus Reconstruction Error: ", final_error)

    # Plotting the consensus reconstruction errors over iterations
    plt.plot(consensus_errors, label=f'Consensus Error (dt={dt}, np={n_particles}, α={alpha}, λ={lambda_}, σ={sigma})')

# Final plot formatting (if multiple runs are plotted)
plt.xlabel('Iteration')
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error of Consensus Point Over Iterations (Grid Search)')
plt.legend()
plt.grid(True)
plt.show()

# Convert results to a pandas DataFrame
df_results = pd.DataFrame(results)"""

'results = []\n# Create itertools.product to generate all combinations of parameters\nparam_grid = list(itertools.product(dt, n_particles, alpha, lambda_, sigma))\n\n# Iterate over the parameter grid\nfor params in param_grid:\n    dt, n_particles, alpha, lambda_, sigma = params\n\n    n_iterations = int(T / dt) # Varying parameter: Number of iterations\n\n    # Step 2: Initialize\n    particles = []\n    for _ in range(n_particles):\n        A = np.random.randn(I, rank) # n_particles Ixrank matrices\n        B = np.random.randn(J, rank) # n_particles Jxrank matrices\n        C = np.random.randn(K, rank) # n_particles Kxrank matrices\n        particles.append({\'A\': A, \'B\': B, \'C\': C})\n\n    # Print all parameters including fixed ones\n    print(f"Running grid search with params: dt={dt}, n_particles={n_particles}, alpha={alpha}, lambda_={lambda_}, sigma={sigma}, T={T}, I={I}, J={J}, K={K}, rank={rank}, #iters={n_iterations}")\n\n    # List to store reconstruction errors of the c